In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt

In [2]:
#Read in the original csv file
AsheCits = pd.read_csv('../data/Asheville_Citations.csv', low_memory=False)

#Data Source: https://data-avl.opendata.arcgis.com/datasets/05af48ba4caf43499a37f348abb98a66_0/explore

## Important to note that this includes Driving/Traffic citations.  Pittsburgh dataset does not.  

In [25]:
AsheCits #6494788 rows

,OBJECTID,citation_date,citation_time,address,citation_type,subject_race,subject_gender,agency,tcmainid,objectid_1
1479629,13138681,2012-01-01,2207,100-BLK DEAVERVIEW RD/PATTON AVE,EXPIRED REGISTRATION CARD/TAG ...,W,M,APD,182950,116122
5932267,17591319,2012-01-01,558,1100-BLK PATTON AVE,CONSUME ALC BY 19/20 ...,W,M,APD,182948,117297
2278799,13937851,2012-01-01,558,1100-BLK PATTON AVE,CONSUME ALC BY 19/20 ...,W,M,APD,182948,117313
232331,11891383,2012-01-01,558,1100-BLK PATTON AVE,CONSUME ALC BY 19/20 ...,W,M,APD,182948,117315
694132,12353184,2012-01-01,558,1100-BLK PATTON AVE,CONSUME ALC BY 19/20 ...,W,M,APD,182948,117315
...,...,...,...,...,...,...,...,...,...,...
5229897,16888949,2020-11-30,1508,I 40/SMOKEY PARK HWY,SOLICITING FROM HIGHWAY,B,M,APD,290534,196889
4584593,16243645,2020-11-30,1551,ACTON CIRCLE/127 ACTON CIRCLE,MISDEMEANOR LARCENY,W,F,APD,290533,197115
6011898,17670950,2020-11-30,1928,I 240/MONTFORD ST EXIT,SPEEDING,NaN,M,APD,290536,198619
5931416,17590468,2020-11-30,1443,ACTON CIRCLE/ACTON WOODS RD,NO OPERATORS LICENSE,W,F,APD,290538,198618


In [26]:
#convert times to datetimes and sort
AsheCits['citation_date'] = pd.to_datetime(AsheCits['citation_date'])
AsheCits = AsheCits.sort_values(['citation_date'], ascending=True)
#Date range covered: 2012-01-01 - 2020-11-30
AsheCits.tail()

,OBJECTID,citation_date,citation_time,address,citation_type,subject_race,subject_gender,agency,tcmainid,objectid_1
4666552,16325604,2020-11-30,1928,I 240/MONTFORD ST EXIT,SPEEDING,NaN,M,APD,290536,198611
4666551,16325603,2020-11-30,1443,ACTON CIRCLE/ACTON WOODS RD,NO OPERATORS LICENSE,W,F,APD,290538,198610
6010403,17669455,2020-11-30,1551,ACTON CIRCLE/127 ACTON CIRCLE,MISDEMEANOR LARCENY,W,F,APD,290533,197115
5473013,17132065,2020-11-30,1443,ACTON CIRCLE/ACTON WOODS RD,NO OPERATORS LICENSE,W,F,APD,290538,198607
5390646,17049698,2020-11-30,1649,I 240/I40W,SPEEDING,W,M,APD,290555,196701


In [69]:
#Periodical Cicadas are confirmed in Asheville for Brood VI (2017).  Reducing dataset to relevenat months of 2016-2018
AsheCitstrim = AsheCits[AsheCits['citation_date'] > dt.datetime(2015,4,30)]
AsheCitstrim = AsheCitstrim[AsheCitstrim['citation_date'] < dt.datetime(2019,7,1)]

In [39]:
AsheCitstrim.info() #2544009

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544009 entries, 1827153 to 211651
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   OBJECTID        int64         
 1   citation_date   datetime64[ns]
 2   citation_time   int64         
 3   address         object        
 4   citation_type   object        
 5   subject_race    object        
 6   subject_gender  object        
 7   agency          object        
 8   tcmainid        int64         
 9   objectid_1      int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 213.5+ MB


## Noticed that the tcmainid is duplicated a lot.  I'm going to reduce the dataset to one row per tcmainID

In [71]:
#df = df.drop_duplicates(subset=['Id'])
AsheCitstrim = AsheCitstrim.drop_duplicates(subset=['tcmainid'])

In [51]:
AsheCitstrim = AsheCitstrim.drop_duplicates(subset=['OBJECTID'])

In [41]:
AsheCitstrim.info() #26558 rows, seems more reasonable and checks out when spot-checking in Excel

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26558 entries, 1827153 to 4809735
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OBJECTID        26558 non-null  int64         
 1   citation_date   26558 non-null  datetime64[ns]
 2   citation_time   26558 non-null  int64         
 3   address         26558 non-null  object        
 4   citation_type   26558 non-null  object        
 5   subject_race    25154 non-null  object        
 6   subject_gender  26540 non-null  object        
 7   agency          26558 non-null  object        
 8   tcmainid        26558 non-null  int64         
 9   objectid_1      26558 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 2.2+ MB


## Getting the relevant years

In [72]:
#2015
AsheCits2015 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2015,4,30)]
AsheCits2015 = AsheCits2015[AsheCits2015['citation_date'] < dt.datetime(2015,7,1)]
AsheCits2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2157 entries, 1827153 to 2383413
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OBJECTID        2157 non-null   int64         
 1   citation_date   2157 non-null   datetime64[ns]
 2   citation_time   2157 non-null   int64         
 3   address         2157 non-null   object        
 4   citation_type   2157 non-null   object        
 5   subject_race    2051 non-null   object        
 6   subject_gender  2156 non-null   object        
 7   agency          2157 non-null   object        
 8   tcmainid        2157 non-null   int64         
 9   objectid_1      2157 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 185.4+ KB


In [55]:
#2016
#154981 rows!?! Seems like a lot.  (IT WAS - removed duplicates later)
AsheCits2016 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2016,4,30)]
AsheCits2016 = AsheCits2016[AsheCits2016['citation_date'] < dt.datetime(2016,7,1)]

In [56]:
AsheCits2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 113386 to 399959
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OBJECTID        1625 non-null   int64         
 1   citation_date   1625 non-null   datetime64[ns]
 2   citation_time   1625 non-null   int64         
 3   address         1625 non-null   object        
 4   citation_type   1625 non-null   object        
 5   subject_race    1542 non-null   object        
 6   subject_gender  1624 non-null   object        
 7   agency          1625 non-null   object        
 8   tcmainid        1625 non-null   int64         
 9   objectid_1      1625 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 139.6+ KB


In [57]:
#2017
#926666 rows!?! Holy cow
AsheCits2017 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2017,4,30)]
AsheCits2017 = AsheCits2017[AsheCits2017['citation_date'] < dt.datetime(2017,7,1)]

In [58]:
AsheCits2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1204 entries, 5364381 to 5441729
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OBJECTID        1204 non-null   int64         
 1   citation_date   1204 non-null   datetime64[ns]
 2   citation_time   1204 non-null   int64         
 3   address         1204 non-null   object        
 4   citation_type   1204 non-null   object        
 5   subject_race    1146 non-null   object        
 6   subject_gender  1204 non-null   object        
 7   agency          1204 non-null   object        
 8   tcmainid        1204 non-null   int64         
 9   objectid_1      1204 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 103.5+ KB


In [59]:
#2018
#1521834 rows!
AsheCits2018 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2018,4,30)]
AsheCits2018 = AsheCits2018[AsheCits2018['citation_date'] < dt.datetime(2018,7,1)]

In [60]:
AsheCits2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 1614027 to 4809735
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OBJECTID        889 non-null    int64         
 1   citation_date   889 non-null    datetime64[ns]
 2   citation_time   889 non-null    int64         
 3   address         889 non-null    object        
 4   citation_type   889 non-null    object        
 5   subject_race    837 non-null    object        
 6   subject_gender  889 non-null    object        
 7   agency          889 non-null    object        
 8   tcmainid        889 non-null    int64         
 9   objectid_1      889 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 76.4+ KB


In [73]:
#2019
AsheCits2019 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2019,4,30)]
AsheCits2019 = AsheCits2019[AsheCits2019['citation_date'] < dt.datetime(2019,7,1)]

In [74]:
#Bring into a single dataset
years = [AsheCits2015,AsheCits2016,AsheCits2017,AsheCits2018,AsheCits2019]
AsheCits_final = pd.concat(years)

In [75]:
AsheCits_final.info()  #7298 rows, looking much better without these duplicates

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7298 entries, 1827153 to 299401
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OBJECTID        7298 non-null   int64         
 1   citation_date   7298 non-null   datetime64[ns]
 2   citation_time   7298 non-null   int64         
 3   address         7298 non-null   object        
 4   citation_type   7298 non-null   object        
 5   subject_race    6900 non-null   object        
 6   subject_gender  7295 non-null   object        
 7   agency          7298 non-null   object        
 8   tcmainid        7298 non-null   int64         
 9   objectid_1      7298 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 627.2+ KB


In [63]:
AsheCits_final.head(20)

,OBJECTID,citation_date,citation_time,address,citation_type,subject_race,subject_gender,agency,tcmainid,objectid_1
1827153,13486205,2015-05-01,2159,100-BLK N LEXINGTON AVE,EXPIRED REGISTRATION CARD/TAG ...,W,F,APD,207849,146345
4309591,15968643,2015-05-01,2217,NC 81/FAIRVIEW RD,FAIL TO STOP-STEADY RED LIGHT ...,B,M,APD,225652,143159
3028188,14687240,2015-05-01,833,1000-BLK PATTON AVE,FAIL TO STOP-STEADY RED LIGHT ...,W,F,APD,207797,146341
1744260,13403312,2015-05-01,2150,800-BLK NEW HAW CREEK RD/JAKS RIDGE DR,FAIL STOP STOPSIGN/FLSH RED LT ...,W,M,APD,208122,143368
1507845,13166897,2015-05-01,1238,100-BLK AIRPORT RD,MISDEMEANOR LARCENY ...,W,M,APD,207837,146606
4152070,15811122,2015-05-01,1835,1000-BLK PATTON AVE,MISDEMEANOR LARCENY ...,B,F,APD,207799,146041
3507774,15166826,2015-05-01,1008,HILLSIDE STREET/WASHINGTON RD,FAIL STOP STOPSIGN/FLSH RED LT ...,W,F,APD,225586,143488
5099445,16758497,2015-05-01,736,US 25/SPRINGSIDE RD,NO OPERATORS LICENSE ...,NaN,M,APD,224974,146607
1667627,13326679,2015-05-01,1959,3200-BLK SWEETEN CREEK RD,FAILURE TO REDUCE SPEED ...,W,M,APD,208987,146604
4454283,16113335,2015-05-01,2351,100-BLK HAYWOOD ST,POSSESSION ALCOHOLIC BEVERAGE ...,W,M,APD,207859,146629


## What else can we learn about the data?

In [ ]:
#what kinds of citations are there?
AsheCits_final['citation_type'].unique()

## Export

In [ ]:
#This is actually still a huge amount of data.  Going to trim off some columns
#data = data.drop(columns="cases")
AsheCits_final = AsheCits_final.drop(columns='address')

In [ ]:
AsheCits_final = AsheCits_final.drop(columns='tcmainid') #not sure what this even is / not useful

In [ ]:
AsheCits_final = AsheCits_final.drop(columns='objectid_1')

In [ ]:
AsheCits_final = AsheCits_final.drop(columns='agency') #Asheville PD

In [ ]:
# time of day seems out of scope for this investigation but could be revisited.  
#For the record, Periodical cicadas appear to typically sing between 10am and 5pm.
AsheCits_final = AsheCits_final.drop(columns='citation_time')

In [ ]:
#STILL too much data! trimming more columns
AsheCits_final = AsheCits_final.drop(columns=['subject_race'])

In [ ]:
AsheCits_final.head()

In [ ]:
#Save as a csv
AsheCits_final.to_csv('AsheCits_final.csv', index = False, header=True)

## Dataset is too large even after trimming almost everything.  I think I should move the traffic data to a separate df

In [76]:
List_AsheTrafficCits = ['FOLLOWING TOO CLOSELY','DRIVE AFTER CONSUMING < 21',
       'SPEEDING', 'NO OPERATORS LICENSE                                        ',
        'EXPIRED REGISTRATION CARD/TAG','HIT AND RUN - FAIL INFO',
       'FAIL SURR TITLE/REG CARD/TAG',
       'DRIVE WRONG WAY-ONE WAY DOT', 'EXCEEDING POSTED SPEED', 'FAIL STOP STOPSIGN/FLSH RED LT',
        'FAIL MAINTAIN LANE CONTROL',
      'OPERATE VEH NO INS','UNSEALED WINE/LIQ IN PASS AREA','LEARNERS PERMIT VIOLATION >18',
     'FAIL TO REPORT ACCIDENT','MOTORCYCLE FAIL BURN TAILLIGHT',
       'FAILURE TO REDUCE SPEED','IMPROPER LOADING/COVERING VEH',
       'EXCEEDING SAFE SPEED',
       'FAIL STOP STOPSIGN/FLSH RED LT','SPEED IN EXCESS OF 45 MPH',
       'SPEED IN EXCESS OF 65 MPH',
       'RECKLESS DRVG-WANTON DISREGARD',
       'FAIL TO WEAR SEAT BELT-DRIVER',
        'FAIL TO STOP-STEADY RED LIGH',
       'FAILURE TO YIELD',     
       'DRIVE WRONG WAY-ONE WAY ST/RD',
       'UNSAFE MOVEMENT','OPERATE COMM VEH COMM NOL',
       'HIT/RUN LEAVE SCENE PROP DAM',
        'UNSAFE MOVE MC/BC CHNG/LV LANE','MIRROR VIOLATION',
       'SPEED WORK ZN<=80AND<=15 OVER',
       'DRIVE LEFT OF CENTER','SPEED IN EXCESS OF 35 MPH','FICT/ALT TITLE/REG CARD/TAG',
        'SPEED COMPETITION','DRIVE LEFT OF CENTER-PASS RGHT',
       'FAIL TO YIELD FROM PRIVATE DRV',
       'UNSAFE LANE CHANGE','DRIVE WITHOUT TWO HEADLAMPS',
       'TRAFFIC INFRACTION - FREE TEXT',
       'UNSAFE PASSING YELLOW LINE ',
       'DRIVE/ALLOW MV NO REGISTRATION',
        'FAIL TO OBEY TRAFFIC OFFICER',
       'HIT/RUN FAIL STOP PROP DAMAGE',
        'DRIVE WITHOUT TWO HEADLAMPS','MOBILE PHONE VIOL MOTOR VEH',
        'LICENSE PLATE FRAME/COVER VIOL',
         'MOTORCYCLE/MOPED HELMET/PSGR',
        'FICT/ALT TITLE/REG CARD/TAG',
       'FAIL WEAR SEAT BELT-FRONT SEAT','DWLR VIOL LIMITED DRIVE PRIV',
       'DR/ALLOW REG PLATE NOT DISPLAY ',
       'NO LIABILITY INSURANCE','MOTORCYCLE FAIL BURN HEADLAMP',
       'FAIL TO YIELD LEFT TURN',
       'FAILURE TO DIM LIGHTS MEET VEH',
       'FAIL TO NOTIFY DMV ADDR CHANGE',
        'FAIL WEAR SEAT BELT-REAR SEAT',
       'EXCEEDING POSTED SPEED', 'EXPIRED OPERATORS LICENSE                                   ',
       'DIRECTIONAL SIGNALS EQUIP VIOL',
       'BRAKE/STOP LIGHT EQUIP VIOL', 'FICT/CNCL/REV REG CARD/TAG',
       'RECKLESS DRIVING TO ENDANGER','IMPROPER TURN',
        'TEXTING VIOL MOTOR VEH',
       'IMPROPER PASSING ON RIGHT',
       'FAIL PEDESTRIAN RIGHT OF WAY',
        'FAIL TO BURN HEADLAMPS',
       'ILLEGAL RIGHT TURN ON RED','LOW-SPEED VEHICLE VIOLATION',
       'UNSAFE MOVE MC/BC CRASH SBINJ',
       'DRIVE CVEH CLIC DISQUALIFIED','UNAUTHORIZED USE OF MOTOR VEH',
       'OVER LOAD SIZE/LENGTH/ VEHICLE',
       'WINDOW TINTING VIOL',
       'FAIL TO HEED LIGHT OR SIREN','HIT/RUN FAIL STOP PROP DAMAGE                               ',
       'FAILURE TO REDUCE SPEED'
       'UNSAFE TIRES','COVER REG STICKER/PLATE',
       'RECKLESS DRVG-WANTON DISREGARD',
        'HIT/RUN LEAVE SCENE PROP DAM','SPEEDING', 'NO LIABILITY INSURANCE',
       'REAR LAMPS VIOLATION', 'EXCEEDING SAFE SPEED',
       'DRIVING WHILE IMPAIRED', 'AGGRESSIVE DRIVING',
       'EXPIRED REGISTRATION CARD/TAG', 'OPERATE VEH NO INS',
       'FAIL CARRY REGISTRATION CARD', 'DRIVE/ALLOW MV NO REGISTRATION',
    'OVERLOADED/OVERCROWDED VEHICLE',
       'DRIVE WRONG WAY-ONE WAY LOCAL',    
      'LOCATION OF TV IN VEHICLE',
      'FAIL YLD STOPSIGN/FLSH RED LGT',
       'PERMIT OPERATION VEH NO INS','RECKLESS DRIVING TO ENDANGER',
       'UNSAFE LANE CHANGE', 'FAIL TO STOP FOR STOPPED BUS',
       'FAIL TO YIELD TO PEDESTRIAN',
       'AGGRESSIVE DRIVING',
       'UNLAWFULLY PASS EM/PUB SV VEH',
       'DRIVING WHILE IMPAIRED','DWLR NOT IMPAIRED REV                                       ',
       'FAIL EXHIBIT/SURRENDER LICENSE',             
         'NO MOTORCYCLE ENDORSEMENT',    
        'CENTER LANE VIOLATION',
       'SPEED WORK ZN >80 OR>15 OVER',
       'TRAFFIC INFRACTION - FREE TEXT',
       'SPEED IN EXCESS OF 55 MPH',
        'IMPROPER EQUIP - SPEEDOMETER',
        'REAR LAMPS VIOLATION',
        'RECKLESS DRVG W CMV LOAD PRMT',
        'FAIL CARRY VALID DRIVERS LIC',
       'STOP WHERE TRAFFIC OBSTRUCTED',           
       'WHITE LIGHT REAR-DRIVE FORWARD',
       'TRAFFIC OFFENSE - FREE TEXT',
       'TEXTING/EMAIL VIOL MOTOR VEH',
        'NO HEADLIGHTS ON WIPERS ON',
        'ALLOW UNLICENSED TO DRIVE',
        'IMPEDE TRAFFIC BY SLOW SPEED',
        'FAIL TO STOP FOR STOPPED BUS',
       'IMPROPER BRAKES',
        'IMPROPER PASSING',
        'FAIL TO YIELD TO PEDESTRIAN',
        'FAIL TO YIELD EMERG VEH NO ACC',
       'UNSAFE PASSING ONCOMING TRAF',
        'SPEEDING IN SCHOOL ZONE',
        'IMPROPER MUFFLER',
       'CHILD NOT IN REAR SEAT',
       'ADDITIONAL LIGHTING EQUIP VIOL',
                        'HIT/RUN FAIL STOP PROP DAMAGE'
        'IMPROPER VEHICLE INSPECTION',
       'NO REGISTRATION CARD',
       'LIC/PERMIT NO SUPV DRIVER <18',
       'LIC/PERMIT SEATING VIOL <18','DWLR NOT IMPAIRED REV'
       'REG PLAT IMPROPERLY ATTACHED', 
        'FAILURE TO YIELD - YIELD SIGN',
         'WRONG WAY ON DUAL LANE HIGHWAY',     
        'COVERING/DISGUISING REG PLATE',
       'IMPROPER STEERING MECHANISM',
       'USE OF RED OR BLUE LIGHT',
       'GIVE/LEND/BORROW LIC PLATE', 'DWLR',
       'NO OPERATORS LICENSE                                        ',
       'FOLLOWING TOO CLOSELY                                       ',
       'SPEEDING                                                    ',
       'EXPIRED REGISTRATION CARD/TAG                               ',
       'DWLR                                                        ',
       'DWLR IMPAIRED REV                                           ',
       'FAIL MAINTAIN LANE CONTROL                                  ',
       'CENTER LANE VIOLATION                                       ',
       'SPEED WORK ZN >80 OR>15 OVER                                ',
       'TRAFFIC INFRACTION - FREE TEXT                              ',
       'SPEED IN EXCESS OF 55 MPH                                   ',
       'RECKLESS DRVG W CMV LOAD PRMT                               ',
       'WHITE LIGHT REAR-DRIVE FORWARD                              ',
       'TRAFFIC OFFENSE - FREE TEXT                                 ',
       'TEXTING/EMAIL VIOL MOTOR VEH                                ',
       'NO HEADLIGHTS ON WIPERS ON                                  ',
       'DRIVE WITHOUT TWO HEADLAMPS                                 ',
       'FAIL TO STOP FOR STOPPED BUS                                ',
    'HIT/RUN LEAVE SCENE PROP DAM                                ',
    'MOBILE PHONE VIOL MOTOR VE                                  ',
    'EXCEEDING SAFE SPEED                                        ',
    'MOTORCYCLE FAIL BURN TAILLIGHT                              ',
    'FAIL STOP STOPSIGN/FLSH RED LT                               ',
    'FAIL TO HEED LIGHT OR SIREN                                 ',
                        'FAILURE TO REDUCE SPEED                                     ',
                        'FAIL STOP STOPSIGN/FLSH RED LT                              ',
                        'FAIL TO WEAR SEAT BELT-DRIVER                               ',
       'RECKLESS DRVG-WANTON DISREGARD                              ',
                        'FAIL TO STOP-STEADY RED LIGHT                               ',
       'FAILURE TO YIELD                                            ',
                        'DRIVE WRONG WAY-ONE WAY ST/RD                               ',
                        'UNSAFE MOVEMENT                                             ',
       'UNSAFE MOVE MC/BC CHNG/LV LANE                              ',
       'DRIVE LEFT OF CENTER                                        ',
                        'UNSAFE LANE CHANGE                                          ',
       'UNSAFE PASSING YELLOW LINE                                  ',
       'SPEED COMPETITION                                           ',
       'DRIVE/ALLOW MV NO REGISTRATION                              ',
                        'OPERATE VEH NO INS                                          ',
       'FAIL TO REPORT ACCIDENT                                     ',
                        'FAIL TO OBEY TRAFFIC OFFICER                                ',
                        'MOTORCYCLE/MOPED HELMET/PSGR                                ',
       'FAIL TO SECURE PASSEN UNDER 16                              ',
                        'IMPROPER BACKING                                            ',
       'FICT/ALT TITLE/REG CARD/TAG                                 ',
       'FAIL WEAR SEAT BELT-FRONT SEAT                              ',
       'DR/ALLOW REG PLATE NOT DISPLAY                              ',
       'NO LIABILITY INSURANCE                                      ',
                        'FAILURE TO DIM LIGHTS MEET VEH                              ',
       'FAIL TO NOTIFY DMV ADDR CHANGE                              ',
                        'FAIL WEAR SEAT BELT-REAR SEAT                               ',
                        'TEXTING VIOL MOTOR VEH                                      ',
       'IMPROPER PASSING ON RIGHT                                   ',
                        'FAIL TO BURN HEADLAMPS                                      ',
       'ILLEGAL RIGHT TURN ON RED                                   ',
       'UNSAFE TIRES                                                ',
                        'DRIVE WRONG WAY-ONE WAY LOCAL                               ',
                        'OVERLOADED/OVERCROWDED VEHICLE                              ',
                        'AGGRESSIVE DRIVING                                          ',
       'UNLAWFULLY PASS EM/PUB SV VEH                               ',
       'DRIVING WHILE IMPAIRED                                      ',
       'FAIL EXHIBIT/SURRENDER LICENSE                              ',
                        'NO MOTORCYCLE ENDORSEMENT                                   ',
                        'STOP WHERE TRAFFIC OBSTRUCTED                               ',
       'FAIL CARRY VALID DRIVERS LIC                                ',
                        'IMPROPER BRAKES                                             ',
                        'IMPROPER PASSING                                            ',
                        'IMPROPER MUFFLER                                            ',
       'CHILD NOT IN REAR SEAT                                      ',
       'ADDITIONAL LIGHTING EQUIP VIOL                              ',
                        'FAILURE TO YIELD - YIELD SIGN                               ',
                        'LEARNERS PERMIT VIOLATION >18                               ',
                        'UNSAFE MOVE MC/BC CRSH DMG/INJ                              ',
                        'HIT AND RUN - FAIL INFO                                     ',
       'FAIL SURR TITLE/REG CARD/TAG                                ',
                        'IMPROPER USE DEALER PLATES                                  ',
       'PASSENGER FLEE ACC PROP DAM                                 ',
       'IMPROPER LOADING/COVERING VEH                               ',
       'ILLEGAL PARKING                                             ',
                        'PARK IN UNDESIGNATED AREAS                                  ',
       'DRIVE LEFT OF CENTER-PASS RGHT                              ',
                        'OPERATE COMM VEH COMM NOL                                   ',
       'PARK FIRE HYD/STATION/PRIV DR                               ',
       'STOP/LEAVE STAND MV INTER/CAH                               ',
       'LEAVE VEH UNATTENDED/UNSECURE                               ',
       'MIRROR VIOLATION                                            ',
       'SPEED WORK ZN<=80AND<=15 OVER                               ',
       'OBSTRUCT PEDESTRIAN SIDEWALK                                ',
       'MOTORCYCLE FAIL BURN HEADLAMP                               ',
       'UNINTENTIONAL LITTER <= 15 LB                               ',
       'NO LIGHTS ON PARKED VEHICLE                                 ',
       'LOW-SPEED VEHICLE VIOLATION                                 ',
       'UNSAFE MOVE MC/BC CRASH SBINJ                               ',
                        'EXPIRED/NO INSPECTION                                       ',
                        'CANCL/REVOK/SUSP CERTIF/TAG                                 ',
                        'LICENSE PLATE FRAME/COVER VIOL                              ',
                        'FAIL TO YIELD LEFT TURN                                     ',
                        'BRAKE/STOP LIGHT EQUIP VIOL                                 ',
       'EXCEEDING POSTED SPEED                                      ',
       'DIRECTIONAL SIGNALS EQUIP VIOL                              ',
       'RECKLESS DRIVING TO ENDANGER                                ',
                        'IMPROPER EQUIP - SPEEDOMETER                                ',
              'ALLOW UNLICENSED TO DRIVE                                   ',
'IMPEDE TRAFFIC BY SLOW SPEED                                ',
                        'FICT/CNCL/REV REG CARD/TAG                                  ',
                        'IMPROPER TURN                                               ',
                        'FAIL TO YIELD EMERG VEH NO ACC                              ',
       'UNSAFE PASSING ONCOMING TRAF                                ',
                        'SPEEDING IN SCHOOL ZONE                                     ',
       'DRIVE CVEH CLIC DISQUALIFIED                                ',
                        'UNAUTHORIZED USE OF MOTOR VEH                               ',
                         'IMPROPER VEHICLE INSPECTION                                 ',
       'NO REGISTRATION CARD                                        ',
                        'LIC/PERMIT NO SUPV DRIVER <18                               ',
                        'WRONG WAY ON DUAL LANE HIGHWAY                              ',
       'LIC/PERMIT SEATING VIOL <18                                 ',
       'REG PLAT IMPROPERLY ATTACHED                                ',
       'OVER LOAD SIZE/LENGTH/ VEHICLE                              ',
       'WINDOW TINTING VIOL                                         ', 'ILLEGAL PARKING','OBSTRUCT PEDESTRIAN SIDEWALK',
                        'UNSAFE PASSING RR OR INTERSECT','UNSAFE TIRES', 'PARK FIRE HYD/STATION/PRIV DR',
                        'IMPROPER STEERING MECHANISM                                 ',
       'USE OF RED OR BLUE LIGHT                                    ',
       'GIVE/LEND/BORROW LIC PLATE                                  ',
       'UNSAFE MOVE MC/BC CRASH>5,000                               ',
                        'SPEED IN EXCESS OF 35 MPH                                   ',
       'HANDICAPPED PARKING VIOLATION                               ',
                        'SPEED IN EXCESS OF 45 MPH                                   ',
                        'FAIL TO YIELD TO PEDESTRIAN                                 ',
       'SPEED IN EXCESS OF 65 MPH                                   ',
                        'COVER REG STICKER/PLATE                                     ',
                        'POSS OPN CNT/CONS ALC PSG AREA                              ',
                        'UNSEALED WINE/LIQ IN PASS AREA                              ',
                        'SPEED IN EXCESS OF 35 MPH                                   ',
                        'REG CARD ADDRESS CHANGE VIO                                 ',
                        'DRIVE WRONG WAY ON DUAL LANE                                ',
                        'FAIL TO YIELD FROM PRIVATE DRV                              ', 
                        'POSS OPN CNT/CONS ALC PSG AREA',
                        'FAIL YLD STOPSIGN/FLSH RED LGT                              ',
       'PERMIT OPERATION VEH NO INS                                 ',
                        'DRIVE AFTER CONSUMING < 21                                  ','DR/ALLOW REG PLATE NOT DISPLAY',
                        'FAIL TO APPLY FOR NEW TITLE                                 ','UNSAFE MOVE MC/BC CRASH>5,000','DRINK BEER/WINE WHILE DRIVING',
                        'IMPROPER BACKING',
       'DRIVE WRONG WAY-ONE WAY DOT                                 ',
                        'MOBILE PHONE VIOL MOTOR VEH                                 ','FAIL TO SECURE PASSEN UNDER 16',
       'LIC/PRMT VIOL OTH RSTRCTN <18                               ',
                        'FAIL TO STOP-STEADY RED LIGHT',
                        'FAIL TO SECURE PASSEN UNDER 16','HANDICAPPED PARKING VIOLATION',
                        'FAIL PEDESTRIAN RIGHT OF WAY                                ',
                        'DWLR VIOL LIMITED DRIVE PRIV                                ', 'UNSAFE PASSING YELLOW LINE',
                        'UNSAFE MOVE MC/BC CRSH DMG/INJ', 'RECKLESS DRIVING AFT ALC(R/83)','IMPROPER VEHICLE INSPECTION', 'PARK IN UNDESIGNATED AREAS'
           ]

In [77]:
AsheCits_notTraffic = AsheCits_final[(~AsheCits_final['citation_type'].isin(List_AsheTrafficCits))]

In [78]:
AsheCits_notTraffic.info() #2858 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2858 entries, 1507845 to 299401
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OBJECTID        2858 non-null   int64         
 1   citation_date   2858 non-null   datetime64[ns]
 2   citation_time   2858 non-null   int64         
 3   address         2858 non-null   object        
 4   citation_type   2858 non-null   object        
 5   subject_race    2766 non-null   object        
 6   subject_gender  2856 non-null   object        
 7   agency          2858 non-null   object        
 8   tcmainid        2858 non-null   int64         
 9   objectid_1      2858 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 245.6+ KB


In [ ]:
AsheCits_notTraffic['citation_type'].unique()

In [79]:
#How big is this file now?
AsheCits_final = AsheCits_notTraffic

#export
AsheCits_final.to_csv('AsheCits_final.csv', index = False, header=True)